# Commercial Manager Insight Pack
## Sales Performance Follow‑Up Intelligence (Portfolio Case Study)

**Objective:** Transform messy, WhatsApp‑style daily sales submissions into **commercial decision support**.

**Primary user:** Commercial Manager (Distributor / Concession context)

**Core questions this notebook answers:**
1. How are we trading **daily / monthly / yearly**?
2. Who are the **top performers**?
3. How disciplined is reporting (**on‑time vs late vs next‑day**)?
4. Who needs **follow‑up today** (priority list)?

> **Disclaimer:** Data used here is **synthetic** and generated to mimic real operational conditions (late submissions, duplicates, missing fields). No proprietary or personal data was used.


---
## 1) Setup & Data Load
Load the cleaned, analytics‑ready fact table produced by the Python parsing pipeline.

**Expected input file (local):** `data/processed/daily_sales_fact_v2.csv`

**Outputs (saved for GitHub):** charts in `reports/figures/` and a follow‑up CSV in `reports/`.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Paths
DATA = Path("../data/processed/daily_sales_fact_v2.csv")
FIG_DIR = Path("../reports/figures")
FIG_DIR.mkdir(parents=True, exist_ok=True)

# Load
df = pd.read_csv(DATA, parse_dates=["sale_date"])

print("Rows:", len(df))
print("Date range:", df["sale_date"].min().date(), "to", df["sale_date"].max().date())
df.head()


---
## 2) Revenue Trend (Daily / Monthly / Yearly)
Commercial managers need different time horizons:
- **Daily:** detect sudden dips/spikes and day‑of‑week patterns.
- **Monthly:** assess momentum and run‑rate.
- **Yearly:** high‑level executive context.

All figures use **Sum of Revenue (ZAR)**.


In [ ]:
# 2A) Daily revenue trend
daily = df.groupby("sale_date", as_index=False)["revenue"].sum()

plt.figure()
plt.plot(daily["sale_date"], daily["revenue"])
plt.title("Daily Revenue Trend")
plt.xlabel("Sale date")
plt.ylabel("Revenue (ZAR)")
plt.tight_layout()
plt.savefig(FIG_DIR / "01_daily_revenue_trend.png", dpi=200)
plt.show()


In [ ]:
# 2B) Monthly revenue trend
tmp = df.copy()
tmp["month"] = tmp["sale_date"].dt.to_period("M").dt.to_timestamp()
monthly = tmp.groupby("month", as_index=False)["revenue"].sum()

plt.figure()
plt.plot(monthly["month"], monthly["revenue"])
plt.title("Monthly Revenue Trend")
plt.xlabel("Month")
plt.ylabel("Revenue (ZAR)")
plt.tight_layout()
plt.savefig(FIG_DIR / "02_monthly_revenue_trend.png", dpi=200)
plt.show()


In [ ]:
# 2C) Yearly revenue summary
tmp = df.copy()
tmp["year"] = tmp["sale_date"].dt.year
yearly = tmp.groupby("year", as_index=False)["revenue"].sum()

plt.figure()
plt.bar(yearly["year"].astype(str), yearly["revenue"])
plt.title("Yearly Revenue Summary")
plt.xlabel("Year")
plt.ylabel("Revenue (ZAR)")
plt.tight_layout()
plt.savefig(FIG_DIR / "03_yearly_revenue_summary.png", dpi=200)
plt.show()

yearly


---
## 3) Rep Performance Leaderboard
This section identifies top performers and highlights where follow‑up coaching may be needed.


In [ ]:
# Top 15 reps by total revenue
rep_top15 = (df.groupby("rep_key", as_index=False)["revenue"].sum()
             .sort_values("revenue", ascending=False)
             .head(15))

plt.figure()
plt.barh(rep_top15["rep_key"][::-1], rep_top15["revenue"][::-1])
plt.title("Top 15 Reps by Total Revenue")
plt.xlabel("Revenue (ZAR)")
plt.ylabel("Rep")
plt.tight_layout()
plt.savefig(FIG_DIR / "04_rep_leaderboard_top15.png", dpi=200)
plt.show()

rep_top15


---
## 4) Submission Discipline (On‑time vs Late vs Next‑day)
Reporting discipline affects visibility and response time. This view quantifies operational discipline.


In [ ]:
status_counts = (df["submission_status"]
                 .value_counts()
                 .reindex(["on_time", "late", "next_day"])
                 .fillna(0))

plt.figure()
plt.bar(status_counts.index, status_counts.values)
plt.title("Submission Status Counts")
plt.xlabel("Submission status")
plt.ylabel("Count of records")
plt.tight_layout()
plt.savefig(FIG_DIR / "05_submission_status_counts.png", dpi=200)
plt.show()

status_counts


---
## 5) Follow‑Up Priority View (Top 25)
**Purpose:** provide a daily action list for a commercial manager.

Priority is higher when:
- submission is **next‑day** (visibility delay)
- submission is **late**
- there are **data quality issues** (`parse_status`)

> Note: The scoring is intentionally simple and explainable.


In [ ]:
followup = df[
    (df["submission_status"].isin(["late", "next_day"])) |
    (df["parse_status"].isin(["missing_units", "missing_revenue", "missing_both"]))
].copy()

followup["priority_score"] = 0
followup.loc[followup["submission_status"] == "next_day", "priority_score"] += 2
followup.loc[followup["submission_status"] == "late", "priority_score"] += 1
followup.loc[followup["parse_status"].isin(["missing_units", "missing_revenue", "missing_both"]), "priority_score"] += 1

followup_top25 = (followup
    .sort_values(["priority_score", "revenue"], ascending=[False, False])
    .loc[:, ["sale_date", "region", "store", "rep_key", "revenue", "units_sold",
             "submission_status", "parse_status", "priority_score"]]
    .head(25)
)

followup_top25


In [ ]:
# Save follow-up list for sharing / review
OUT = Path("../reports/follow_up_priority_top25.csv")
OUT.parent.mkdir(parents=True, exist_ok=True)
followup_top25.to_csv(OUT, index=False)
print("Saved:", OUT)


---
## 6) Portfolio Notes
**What this demonstrates:**
- Realistic commercial reporting constraints (late submissions, duplicates)
- A reproducible pipeline: raw → parsed → fact table
- Decision‑first analytics (follow‑up prioritization)

**Next optional step:** replicate these views in Power BI for an interactive dashboard.
